In [7]:
# %reset
## This code builds an ANN surrogate model for two-terminal connectivity.
## Written by Mohammad Amin Nabian, mnabia2@illinois.edu, March 2017

## Import libraries
import numpy as np
import networkx as nx
import random
import copy
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from sklearn import cross_validation
from keras.layers.core import Dense, Activation, Dropout
from keras.utils import np_utils, generic_utils
from keras.models import Sequential
from keras.models import model_from_json
from keras import optimizers

## Initialization
nbridges=39

## Generating the network topology
G=nx.Graph()
G.add_node(0)
G.add_node(1)
G.add_node(2)
G.add_node(3)
G.add_node(4)
G.add_node(5)
G.add_node(6)
G.add_node(7)
G.add_node(8)
G.add_node(9)
G.add_node(10)
G.add_node(11)
G.add_edge(0,1)
G.add_edge(0,2)
G.add_edge(1,3)
G.add_edge(1,4)
G.add_edge(2,3)
G.add_edge(3,4)
G.add_edge(2,6)
G.add_edge(3,4)
G.add_edge(3,5)
G.add_edge(4,7)
G.add_edge(6,7)
G.add_edge(5,6)
G.add_edge(5,8)
G.add_edge(6,9)
G.add_edge(7,8)
G.add_edge(7,11)
G.add_edge(8,10)
G.add_edge(8,9)
G.add_edge(9,10)
G.add_edge(10,11)

## Assign the failure probabilities
s=(nbridges,2);b=np.zeros(s)
b[0]=(0,1)
b[1]=(0,2)
b[2]=(0,2)
b[3]=(0,2)
b[4]=(0,2)
b[5]=(0,2)
b[6]=(0,2)
b[7]=(1,3)
b[8]=(1,4)
b[9]=(1,4)
b[10]=(2,3)
b[11]=(2,3)
b[12]=(2,3)
b[13]=(2,3)
b[14]=(2,3)
b[15]=(2,3)
b[16]=(2,6)
b[17]=(2,6)
b[18]=(2,6)
b[19]=(2,6)
b[20]=(3,4)
b[21]=(3,4)
b[22]=(3,4)
b[23]=(3,5)
b[24]=(3,5)
b[25]=(4,7)
b[26]=(5,6)
b[27]=(5,6)
b[28]=(7,8)
b[29]=(7,11)
b[30]=(7,11)
b[31]=(7,11)
b[32]=(7,11)
b[33]=(8,10)
b[34]=(8,10)
b[35]=(9,10)
b[36]=(9,10)
b[37]=(9,10)
b[38]=(9,10)

# Import the data for surrogate
fpb = np.loadtxt('SURVIVALS_Train.txt')
nsamplesEQ = np.shape(fpb)[1]
nsamplesMC = 10
fpb_samples = np.zeros((nbridges,nsamplesEQ,nsamplesMC),dtype=int)
for i in range (nbridges):
    for j in range (nsamplesEQ):
        for k in range (nsamplesMC):
            if (np.random.rand() < 1. - fpb[i,j]):
                fpb_samples[i,j,k] = 1   # bridge will fail

def CONNECTIVITY_train (index):
    connect_flag = np.zeros(nsamplesMC)
    for p in range (nsamplesMC):
        T=copy.deepcopy(G)
        for i in range (nbridges):
            if fpb_samples[i,index,p] == 1:
                if T.has_edge(int(b[i,0]),int(b[i,1])):
                    T.remove_edge(*b[i])
        if nx.has_path(T,0,11):
            connect_flag[p] = 1
    del T
    return connect_flag

connect_matrix = Parallel(n_jobs=8)(delayed(CONNECTIVITY_train)(index) for index in range(nsamplesEQ))
# np.shape(connect_matrix) = (nsamplesEQ,nsamplesMC)

# Reshape
X = np.zeros((nsamplesEQ*nsamplesMC,nbridges),dtype=int)
for i in range (nbridges):
    c = -1
    for j in range (nsamplesEQ):
        for k in range (nsamplesMC):
            c += 1
            X[c,i] = fpb_samples[i,j,k]
            
Y = np.zeros((nsamplesEQ*nsamplesMC,1),dtype=int)
c = -1
for j in range (nsamplesEQ):
    for k in range (nsamplesMC):
        c += 1
        Y[c] = connect_matrix[j][k]

# Perform the deep learning
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, Y, test_size=0.2)
y_train, y_test = [np_utils.to_categorical(x) for x in (y_train, y_test)]
model = Sequential()
model.add(Dense(output_dim=128, input_shape=(nbridges,)))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(output_dim=64))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(output_dim=32))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(output_dim=16))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(output_dim=8))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(output_dim=4))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(output_dim=2))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, nb_epoch=150, batch_size=64)
deep_score = model.evaluate(X_test, y_test, batch_size=64)
print('The score of your regression is')
print(deep_score)

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
np.savetxt('samples_train.txt',X_train)
np.savetxt('samples_test.txt',X_test)

Epoch 1/150
16000/16000 [==============================] - 0s - loss: 0.5808 - acc: 0.7316     
Epoch 2/150
16000/16000 [==============================] - 0s - loss: 0.3958 - acc: 0.8574     
Epoch 3/150
16000/16000 [==============================] - 0s - loss: 0.3338 - acc: 0.8827     
Epoch 4/150
16000/16000 [==============================] - 0s - loss: 0.2894 - acc: 0.8981     
Epoch 5/150
16000/16000 [==============================] - 0s - loss: 0.2431 - acc: 0.9175     
Epoch 6/150
16000/16000 [==============================] - 0s - loss: 0.2123 - acc: 0.9308     
Epoch 7/150
16000/16000 [==============================] - 0s - loss: 0.1861 - acc: 0.9384     
Epoch 8/150
16000/16000 [==============================] - 0s - loss: 0.1680 - acc: 0.9449     
Epoch 9/150
16000/16000 [==============================] - 0s - loss: 0.1403 - acc: 0.9567     
Epoch 10/150
16000/16000 [==============================] - 0s - loss: 0.1309 - acc: 0.9595     
Epoch 11/150
16000/16000 [=============